<a href="https://colab.research.google.com/github/abhranil-datascience/ClassificationPOC/blob/master/SVMClassificationPOC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
############################## Mount Drive ######################################## 
from google.colab import drive
drive.mount('/content/gdrive')
############################## Change Directory ###################################
import os
os.chdir('/content/gdrive/My Drive/MLandDLFullCourse/ML/Classification/SVM')
########## Suppress Warnings ############
import warnings
warnings.filterwarnings('ignore')
########### Import Dataset #############
import pandas as pd
dataset=pd.read_csv('Social_Network_Ads.csv')
X=dataset.iloc[:,1:4].values
Y=dataset.iloc[:,4].values

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
########### Categorical Variable ###############
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()
X[:,0]=le.fit_transform(X[:,0])
########## Feature Scaling ##############
from sklearn.preprocessing import MinMaxScaler
SC=MinMaxScaler(feature_range=(0,1))
XScaled=SC.fit_transform(X)
########## Train Test Split #############
from sklearn.model_selection import train_test_split
XTrain,XTest,YTrain,YTest=train_test_split(XScaled,Y,test_size=0.1,random_state=42)
########## Check VIF ####################
from statsmodels.stats.outliers_influence import variance_inflation_factor
VIF=[0]*XTrain.shape[1]
for col in range(0,XTrain.shape[1]):
  VIF[col]=VIF[col]+variance_inflation_factor(exog=XTrain,exog_idx=col)#1.6142518559270171, 2.9082305467018816, 2.753146445004674
VIF

[1.6142518559270171, 2.9082305467018816, 2.753146445004674]

In [0]:
########## Grid Search CV for hyperparameter tuning ##########
from sklearn.svm import SVC
RegressorSVC_GS=SVC()
parametersGS=[{'C':[1],
              'kernel':['poly'],
              'degree':[3],
              'gamma':[0.9],
              'coef0':[0.5],
              'shrinking':[True],
              'probability':[True],
              'tol':[0.00001,0.00005,0.0001],
              'decision_function_shape':['ovo','ovr'],
              'verbose':[2]}]
from sklearn.model_selection import GridSearchCV
GS=GridSearchCV(estimator=RegressorSVC_GS,param_grid=parametersGS,scoring='accuracy',cv=3)
GS.fit(XTrain,YTrain)
best_params_SVC=GS.best_params_
best_params_SVC

[LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM]

{'C': 1,
 'coef0': 0.5,
 'decision_function_shape': 'ovo',
 'degree': 3,
 'gamma': 0.9,
 'kernel': 'poly',
 'probability': True,
 'shrinking': True,
 'tol': 1e-05,
 'verbose': 2}

In [0]:
######### KFold Cross Validation #############
RegressorSVC=SVC(C=1,coef0=0.5,decision_function_shape='ovo',degree=3,gamma=0.9,kernel='poly',
                 probability=True,shrinking=True,tol=0.00001)
from sklearn.model_selection import KFold
from sklearn import metrics as mt
import statistics as st
Accuracy=[]
kf=KFold(n_splits=40,shuffle=True)
for count in range(0,100):
  for train_index,val_index in kf.split(XTrain):
    XTrainKF,YTrainKF=XTrain[train_index],YTrain[train_index]
    XValKF,YValKF=XTrain[val_index],YTrain[val_index]
    RegressorSVC.fit(XTrainKF,YTrainKF)
    YPRedKF=RegressorSVC.predict(XValKF)
    Accuracy.append(mt.accuracy_score(YValKF,YPRedKF))
print("Average Accuracy: "+str(st.mean(Accuracy)))#0.8846388888888889

Average Accuracy: 0.8844444444444445


In [0]:
########### Predict ###############
RegressorSVC.fit(XTrain,YTrain)
YPred=RegressorSVC.predict(XTest)
cm=mt.confusion_matrix(YTest,YPred)
cm

array([[26,  1],
       [ 1, 12]])